In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets


**ARCHITECTURE 1**

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [4]:
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:06<00:00, 26628847.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return (correct / total) * 100

In [7]:
class ARCHITECTURE_1(nn.Module):
    def __init__(self, num_classes=10):
        super(ARCHITECTURE_1, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [9]:
model_1= ARCHITECTURE_1().to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_1.parameters(), lr=0.01, momentum=0.9)

In [11]:
num_epochs = 10
for epoch in range(num_epochs):
    model_1.train()
    running_loss = 0.0
    running_acc = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model_1(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_acc += accuracy(outputs, labels) * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_acc / len(train_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


Epoch [1/10], Loss: 1.9324, Accuracy: 29.10%
Epoch [2/10], Loss: 1.6358, Accuracy: 40.00%
Epoch [3/10], Loss: 1.5034, Accuracy: 44.92%
Epoch [4/10], Loss: 1.4404, Accuracy: 47.45%
Epoch [5/10], Loss: 1.3691, Accuracy: 50.62%
Epoch [6/10], Loss: 1.3307, Accuracy: 52.10%
Epoch [7/10], Loss: 1.2971, Accuracy: 53.33%
Epoch [8/10], Loss: 1.2558, Accuracy: 54.95%
Epoch [9/10], Loss: 1.2312, Accuracy: 56.16%
Epoch [10/10], Loss: 1.2097, Accuracy: 56.65%


In [12]:
model_1.eval()

LeNet5(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Tanh()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [23]:
class ARCHITECTURE_2(nn.Module):
    def __init__(self, num_classes=10):
        super(ARCHITECTURE_2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [26]:
model_2= ARCHITECTURE_2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_2.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10
for epoch in range(num_epochs):
    model_2.train()
    running_loss = 0.0
    running_acc = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model_2(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        running_acc += accuracy(outputs, labels) * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_acc / len(train_dataset)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")


Epoch [1/10], Loss: 2.0594, Accuracy: 22.56%
Epoch [2/10], Loss: 1.6748, Accuracy: 37.85%
Epoch [3/10], Loss: 1.5216, Accuracy: 43.79%
Epoch [4/10], Loss: 1.4181, Accuracy: 48.38%
Epoch [5/10], Loss: 1.3305, Accuracy: 51.96%
Epoch [6/10], Loss: 1.2796, Accuracy: 54.20%
Epoch [7/10], Loss: 1.2396, Accuracy: 55.59%
Epoch [8/10], Loss: 1.2007, Accuracy: 57.25%
Epoch [9/10], Loss: 1.1676, Accuracy: 58.65%
Epoch [10/10], Loss: 1.1536, Accuracy: 58.73%


**ARCHITECTURE 3**